In [ ]:
%pip install pandas
%pip install numpy
%pip install scikit-learn
%pip install xgboost


In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

%cd data/Mens/Season/
ordinal_df = pd.read_csv('2024/MMasseyOrdinals_2024.csv')
games_df = pd.read_csv('2024/MRegularSeasonDetailedResults_2024.csv')


In [ ]:
ordinal_df


In [ ]:
games_df['Week'] = ((games_df['DayNum']-1)/7 +1)
games_df['Week'] = games_df['Week'].apply(np.floor)


ordinal_df['Week'] = ((ordinal_df['RankingDayNum']-1)/7 +1)
ordinal_df['Week'] = ordinal_df['Week'].apply(np.floor)
games_df


In [ ]:
def calculate_additional_stats(df):
    """
    Adds calculated statistics for two-point field goals to the DataFrame.

    Parameters:
    - df (DataFrame): The original game results DataFrame.

    Returns:
    - DataFrame: The modified DataFrame with additional stats.
    """
    df['WFGM2'] = df['WFGM'] - df['WFGM3']
    df['WFGA2'] = df['WFGA'] - df['WFGA3']
    df['LFGM2'] = df['LFGM'] - df['LFGM3']
    df['LFGA2'] = df['LFGA'] - df['LFGA3']
    return df

In [ ]:
def prepare_team_stats(df):
    """
    Prepares and aggregates team statistics and statistics against from game results.

    Parameters:
    - df (DataFrame): The game results DataFrame with additional stats.

    Returns:
    - DataFrame: A DataFrame with average stats per team and stats against.
    """
    df = calculate_additional_stats(df)
    # Stats when the team wins
    win_stats = df[['WTeamID','DayNum','Week', 'WFGM', 'WFGA', 'WFGM2', 'WFGA2', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']].copy()
    win_stats.columns = ['TeamID','DayNum','Week', 'FGM', 'FGA', 'FGM2', 'FGA2', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']
    
    # Stats against the team when it wins (opponents' performance)
    win_against_stats = df[['WTeamID','DayNum','Week', 'LFGM', 'LFGA', 'LFGM2', 'LFGA2', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']].copy()
    win_against_stats.columns = ['TeamID','DayNum','Week', 'FGMA', 'FGAA', 'FGM2A', 'FGA2A', 'FGM3A', 'FGA3A', 'FTMA', 'FTAA', 'ORA', 'DRA', 'AstA', 'TOA', 'StlA', 'BlkA', 'PFA']

    # Stats when the team loses
    lose_stats = df[['LTeamID','DayNum','Week', 'LFGM', 'LFGA', 'LFGM2', 'LFGA2', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']].copy()
    lose_stats.columns = ['TeamID', 'DayNum','Week','FGM', 'FGA', 'FGM2', 'FGA2', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']
    
    # Stats against the team when it loses (opponents' performance)
    lose_against_stats = df[['LTeamID','DayNum','Week','WFGM', 'WFGA', 'WFGM2', 'WFGA2', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']].copy()
    lose_against_stats.columns = ['TeamID','DayNum','Week', 'FGMA', 'FGAA', 'FGM2A', 'FGA2A', 'FGM3A', 'FGA3A', 'FTMA', 'FTAA', 'ORA', 'DRA', 'AstA', 'TOA', 'StlA', 'BlkA', 'PFA']

    # Combine winning and losing stats
    all_stats = pd.concat([win_stats, lose_stats]).sort_values(by=['TeamID', 'DayNum'])
    all_against_stats = pd.concat([win_against_stats, lose_against_stats]).sort_values(by=['TeamID', 'DayNum'])

    Week_DayNum_for =  all_stats[['TeamID', 'DayNum', 'Week']].reset_index(drop=True)
    Week_DayNum_against = all_against_stats[['TeamID', 'DayNum', 'Week']].reset_index(drop=True)

    all_stats.drop(columns=['Week', 'DayNum'], inplace=True)
    all_against_stats.drop(columns=['Week', 'DayNum'], inplace=True)

    stats_rolling = all_stats.groupby('TeamID').rolling(window=10, min_periods=1).mean().reset_index(drop=True)
    against_rolling = all_against_stats.groupby('TeamID').rolling(window=10, min_periods=1).mean().reset_index(drop=True)

    stats_rolling = pd.concat([Week_DayNum_for, stats_rolling], axis=1)
    # print(stats_rolling.columns)
    # print(stats_rolling.head(29))
    against_rolling = pd.concat([Week_DayNum_against, against_rolling], axis=1)

    # # Calculate the mean for stats and stats against separately
    # avg_stats = all_stats.groupby('TeamID').mean().reset_index()
    # avg_against_stats = all_against_stats.groupby('TeamID').mean().reset_index()

    merged_stats = pd.merge(stats_rolling, against_rolling, on=['TeamID','DayNum','Week'], suffixes=('', '_A'))
    return merged_stats

In [ ]:


teams_stats_weekly_df = prepare_team_stats(games_df)
# ordinal_df = ordinal_df.sort_values(by=['TeamID', 'RankingDayNum']).reset_index(drop=True)
# ordinal_df = ordinal_df.rename(columns={'RankingDayNum':'DayNum'})

# system_names = ordinal_df['SystemName'].unique()
# teams_names = ordinal_df['TeamID'].unique()
# system_no_rank_all_teams = []

# for system in system_names:
#     teams_in_system = ordinal_df[ordinal_df['SystemName'] == system]['TeamID'].unique()
#     if len(teams_in_system) != len(teams_names):
#         system_no_rank_all_teams.append(system)

# ordinal_df = ordinal_df[~ordinal_df['SystemName'].isin(system_no_rank_all_teams)]
# ordinal_pivot = ordinal_df.pivot_table(index=['TeamID', 'DayNum', 'Week'], columns='SystemName', values='OrdinalRank').reset_index()
# ordinal_pivot.sort_values(by=['TeamID', 'DayNum'])
# ordinal_pivot = ordinal_pivot.ffill()
# ordinal_pivot = ordinal_pivot.groupby('TeamID').apply(lambda x: x.interpolate(method='linear', limit_direction='both')).reset_index(drop=True)

ordinal_df = ordinal_df.sort_values(by=['TeamID']).reset_index(drop=True)
ordinal_df = ordinal_df.drop(columns=['RankingDayNum'])

system_names = ordinal_df['SystemName'].unique()
teams_names = ordinal_df['TeamID'].unique()
system_no_rank_all_teams = []

for system in system_names:
    teams_in_system = ordinal_df[ordinal_df['SystemName'] == system]['TeamID'].unique()
    if len(teams_in_system) != len(teams_names):
        system_no_rank_all_teams.append(system)

ordinal_df = ordinal_df[~ordinal_df['SystemName'].isin(system_no_rank_all_teams)]
ordinal_pivot = ordinal_df.pivot_table(index=['TeamID'], columns='SystemName', values='OrdinalRank').reset_index()
ordinal_pivot.sort_values(by=['TeamID'])
ordinal_pivot = ordinal_pivot.groupby('TeamID').mean().reset_index()


ordinal_pivot.columns

# ordinal_pivot[(ordinal_pivot['TeamID'] == 1104)]


In [ ]:
games_df[(games_df['WTeamID'] == 1104)]

In [ ]:
"""
This code performs the following operations:

1. Sorts the 'ordinal_pivot' DataFrame by 'TeamID' and 'DayNum' columns.
2. Sorts the 'teams_stats_weekly_df' DataFrame by 'TeamID' and 'DayNum' columns.
3. Merges the sorted 'teams_stats_weekly_df' and 'ordinal_pivot' DataFrames using 'DayNum' and 'TeamID' columns in a backward direction.
4. Sorts the resulting DataFrame by 'TeamID' and 'DayNum' columns and resets the index.
5. Extracts the columns starting from the 35th column and assigns them to 'rank_columns' variable.
6. Fills the missing values in 'rank_columns' for each 'TeamID' using backward filling.
7. Filters the resulting DataFrame to include only rows where 'TeamID' is equal to 1104.

Parameters:
    - ordinal_pivot: DataFrame containing ordinal rankings.
    - teams_stats_weekly_df: DataFrame containing weekly team statistics.

Returns:
    - weekly_stats_w_rating: DataFrame with sorted and merged data, filled with missing values, and filtered by 'TeamID' 1104.
"""
ordinal_pivot = ordinal_pivot.sort_values(by=['TeamID'])

teams_stats_weekly_df = teams_stats_weekly_df.sort_values(by=['TeamID'])
weekly_stats_w_rating = pd.merge(teams_stats_weekly_df, ordinal_pivot, on='TeamID', suffixes=('', '_A'))
weekly_stats_w_rating = weekly_stats_w_rating.sort_values(by=['TeamID']).reset_index(drop=True)
rank_columns = weekly_stats_w_rating.columns[34:]

weekly_stats_w_rating[rank_columns] = weekly_stats_w_rating.groupby('TeamID')[rank_columns].bfill()
weekly_stats_w_rating = weekly_stats_w_rating.dropna(axis = 1, how= 'any')
weekly_stats_w_rating



In [ ]:
def prepare_matchup_data(games_df, stats):
    """
    Merges game data with team stats to prepare matchup data.

    Parameters:
    - games_df (DataFrame): The DataFrame containing game results.
    - avg_stats (DataFrame): The DataFrame containing average stats per team.

    Returns:
    - DataFrame: Matchup data with team stats and game outcome.
    """
    stats.set_index('TeamID', inplace=True)
    processed_data = []

    for _, row in games_df.iterrows():
        team_1, team_2 = sorted((row['WTeamID'], row['LTeamID']))
        team_1_won = 1 if team_1 == row['WTeamID'] else 0
        team_1_stats = stats.loc[team_1].add_prefix('team_1_').iloc[-1]
        team_2_stats = stats.loc[team_2].add_prefix('team_2_').iloc[-1]
        
        matchup_data = {
            'Season': row['Season'],
            'DayNum': row['DayNum'],
            'team_1': team_1,
            'team_2': team_2,
            'team_1_won': team_1_won
        }
        matchup_data.update(team_1_stats)
        matchup_data.update(team_2_stats)

        processed_data.append(matchup_data)

    return pd.DataFrame(processed_data)

In [ ]:
test = prepare_matchup_data(games_df, weekly_stats_w_rating)

In [ ]:
test.isna().sum().drop_duplicates()
test

In [ ]:

def fit_model_scalar(model_param, model_name, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3270)

    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Fit the model
    model_param.fit(X_train_scaled, y_train)
    y_pred = model_param.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'{model_name} scalar accuracy: {accuracy:.2f}')

In [ ]:
def fit_model(model_param, model_name, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3270)

    # sfs = SequentialFeatureSelector(model_param, n_features_to_select=10)
    # sfs.fit(X_train, y_train)
    # X_train = sfs.transform(X_train)
    # X_test = sfs.transform(X_test)
    model_param.fit(X_train, y_train)
    y_pred = model_param.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'{model_name} accuracy: {accuracy:.2f}')

In [ ]:
# matchups = prepare_matchup_data(games_df, weekly_stats_w_rating)

X = test.drop(['DayNum','team_1_won'], axis=1)
y = test['team_1_won']

models = {
    'Decision Tree': DecisionTreeClassifier(random_state=3270, max_depth=10),
    'Random Forest': RandomForestClassifier(random_state=3270, n_estimators=200, max_depth=10, min_samples_split=10),
    'Logistic Regression': LogisticRegression(random_state=3270, max_iter=1000, penalty = None, solver = 'lbfgs', ),
    'XGBoost': XGBClassifier(random_state = 3270, n_estimators = 100, max_depth = 3, learning_rate = 0.1, gamma = 0, subsample = 0.8, colsample_bytree = 0.8)
}

for name, model in models.items():
    fit_model_scalar(model, name, X, y)
    fit_model(model, name, X, y)

In [ ]:
lgr = LogisticRegression(random_state=3270, max_iter=1000, penalty = None, solver = 'lbfgs')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3270)

lgr.fit(X_train, y_train)
y_pred = lgr.predict(X_test)
#Show the predictions and the actual values side by side
pd.DataFrame({'Actual': y_test, 'Predicted': y_pred}).tail(50)
#Show the accuracy of the predictions for the last 50 games
accuracy_score(y_test[len(y_test)-200:], y_pred[len(y_test)-200:])
accuracy_score(y_test, y_pred)



In [ ]:
seasons_matchup_avgs_10_data = []
for season in range(2003 , 2025):
    season_games = pd.read_csv(f'{season}/MRegularSeasonDetailedResults_{season}_matchups_avg_w_rating.csv')
    seasons_matchup_avgs_10_data.append(season_games)
    

In [ ]:
def TimeSeriesSplit_by_season(seasons_data):
    """
    Splits the data into training and testing sets by season. Model is trained on all data up to a certain season and tested on the next season until the last season. 

    Parameters:
    - seasons_data (list): A list of DataFrames containing data for each season.

    Returns:
    - list: A list of tuples containing training and testing sets for each season.
    """
    scaler = StandardScaler()
    model = LogisticRegression(random_state=3270, max_iter=1000, penalty = None, solver = 'lbfgs')
    accuracies = []
    for i in range(1, len(seasons_data)):
        print(f'Testing on Season {seasons_data[i]["Season"].unique()[0]}')
        train = pd.concat(seasons_data[:i])
        train = train.dropna(axis = 'columns', how= 'any')
        X_train = train.drop(['Season','DayNum', 'team_1_won'], axis=1)
        y_train = train['team_1_won']
        X_train = scaler.fit_transform(X_train)
        test = seasons_data[i]
        test = test.dropna(axis = 'columns', how= 'any')
        X_test = test.drop(['Season','DayNum', 'team_1_won'], axis=1)
        X_test = scaler.transform(X_test)
        y_test = test['team_1_won']
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        print(f'accuracy: {accuracy:.5f}')

    print(f'Average accuracy: {np.mean(accuracies):.5f}')
    return model
    

In [ ]:
model = TimeSeriesSplit_by_season(seasons_matchup_avgs_10_data)

In [ ]:
seasons_tourney_data = []
for season in range(2003 , 2025):
    season_games = pd.read_csv(f'{season}/MRegularSeasonCompactResults_{season}.csv')
    seasons_tourney_data.append(season_games)
    

In [ ]:
team_stats_season = {}
for season in range(2003 , 2025):
    stats = pd.read_csv(f'{season}/MRegularSeasonDetailedResults_{season}_avg_w_rating.csv')
    team_stats_season[season] = stats

In [ ]:
def build_game_matchups(bracket_team_matchups, team_stats):
    """
    Builds matchup data for the tournament bracket.

    Parameters:
    - bracket_team_matchups (DataFrame): The DataFrame containing the current matchups for a bracket.

    Returns:
    - DataFrame: A DataFrame where each row contains team_1's and team_2's stats for that specific matchup.
    """
    matchups = []
    for _, row in bracket_team_matchups.iterrows():
        team_1 = row['StrongSeed']
        team_2 = row['WeakSeed']

        team_1_data = team_stats[team_stats['TeamID'] == team_1]
        team_2_data = team_stats[team_stats['TeamID'] == team_2]
        
        team_1_data.columns = [f'team_1_{col}' for col in team_1_data.columns]
        team_2_data.columns = [f'team_2_{col}' for col in team_2_data.columns]
        
        team_1_data = team_1_data.reset_index(drop=True)
        team_2_data = team_2_data.reset_index(drop=True)
        matchup_data = pd.concat([team_1_data, team_2_data], axis=1)
        matchup_data['Slot'] = row['Slot']
        matchups.append(matchup_data)
    return pd.concat(matchups, axis=0)

In [ ]:
def preprocess_playins(seeds_df):
    """
    Preprocess the play in teams by removing 'a' and 'b' designations and preparing strong and weak seeds.
    
    Parameters:
    - seeds_df (DataFrame): The DataFrame containing tournament seeds data, including 'Seed', 'TeamID' columns.
    
    Returns:
    - DataFrame: Processed DataFrame with 'StrongSeed' and 'WeakSeed' for play-in teams.
    """
    playin_teams = seeds_df[seeds_df['Seed'].str.contains('a') | seeds_df['Seed'].str.contains('b')].copy()
    playin_teams['Seed'] = playin_teams['Seed'].str.extract('([0-9A-Z]+)')
    playin_teams_match_df = playin_teams.groupby('Seed')['TeamID'].apply(list).reset_index()
    playin_teams_match_df['StrongSeed'] = playin_teams_match_df['TeamID'].apply(lambda x: x[0])
    playin_teams_match_df['WeakSeed'] = playin_teams_match_df['TeamID'].apply(lambda x: x[1])
    playin_teams_match_df.rename(columns={'Seed': 'Slot'}, inplace=True)
    return playin_teams_match_df.drop(columns='TeamID')


In [ ]:
def predict_bracket_winners(bracket_matchups, model, scalar):
    """
    Predict the winners in the lower bracket using a pre-trained model and scaler.
    
    Parameters:
    - bracket_matchups (DataFrame): DataFrame of matchups in the lower bracket, excluding 'Seed' from scaling.
    - model (Model): Pre-trained prediction model.
    - scalar (Scaler): Pre-fitted scaler object for normalizing data.
    
    Returns:
    - DataFrame: Lower bracket DataFrame with an additional column 'team_1_won' indicating the predicted winner.
    """
    bracket_matchups = bracket_matchups.drop(columns= [])
    lower_bracket_scaled = scalar.fit_transform(bracket_matchups.drop(columns=['Slot']))
    bracket_matchups['team_1_won'] = model.predict(lower_bracket_scaled)
    return bracket_matchups


In [ ]:
def update_seeds_with_winners(bracket, seeds_df):
    """
    Update the seeds DataFrame with the winners from the lower bracket predictions.
    
    Parameters:
    - bracket (DataFrame): The lower bracket DataFrame with predictions.
    - seeds_df (DataFrame): The original seeds DataFrame to be updated with current teams seedings.
    
    Returns:
    - DataFrame: Updated seeds DataFrame with winners.
    """
    bracket_winners = {}
    for _, row in bracket.iterrows():
        slot = row['Slot']
        if slot not in bracket_winners:
            bracket_winners[slot] = []
        bracket_winners[slot].append(row['team_1_TeamID'] if row['team_1_won'] == 1 else row['team_2_TeamID'])
    
    for curr_seed, team in bracket_winners.items():
        seeds_df.loc[len(seeds_df.index)] = [curr_seed, team[0]]
    seeds_df.sort_values(by='Seed', inplace=True)
    return seeds_df


In [ ]:
def build_round_matchups(team_seeds, round_slots):
    """
    Build the Tourney round matchups based on seeds and updates team slots.
    
    Parameters:
    - team_seeds (DataFrame): DataFrame containing the seeds and corresponding team IDs.
    - round_slots (DataFrame): DataFrame containing the slots for the tournament matchups.
    
    Returns:
    - DataFrame: First-round matchups with updated team slots based on seeds.
    """

    for index, row in round_slots.iterrows():
        strong_team = team_seeds[(team_seeds['Seed'] == row['StrongSeed'])]['TeamID'].values[0]
        weak_team = team_seeds[(team_seeds['Seed'] == row['WeakSeed'])]['TeamID'].values[0]
        round_slots.at[index, 'StrongSeed'] = strong_team
        round_slots.at[index, 'WeakSeed'] = weak_team

    return round_slots


In [ ]:
tourney_seeds_df = pd.read_csv('2023/MNCAATourneySeeds_2023.csv')
tourney_seeds_df.drop(columns=['Season'], inplace=True)
tourney_slots_df = pd.read_csv('2023/MNCAATourneySlots_2023.csv')

scalar = StandardScaler()

playin_teams_match_df = preprocess_playins(tourney_seeds_df)
bracket_matchups = build_game_matchups(playin_teams_match_df, team_stats_season[2023].groupby('TeamID').last().reset_index())
bracket_matchups.columns
bracket_matchups = predict_bracket_winners(bracket_matchups, model, scalar)
tourney_seeds_df = update_seeds_with_winners(bracket_matchups, tourney_seeds_df)
print('Done with play-ins')


# curr_round_slots = tourney_slots_df[tourney_slots_df['Slot'].str.contains('R1')]

# round_df = build_round_matchups(tourney_seeds_df,curr_round_slots)

# current_round_bracket = build_game_matchups(round_df, team_stats_season[2023])
# current_round_bracket.reset_index(drop=True, inplace=True)
# current_round_bracket = predict_bracket_winners(current_round_bracket, model, scalar)
# tourney_seeds_df = update_seeds_with_winners(current_round_bracket, tourney_seeds_df)


rounds = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6']
# rounds = ['R1']
matchups = []
for current_round in rounds:
    curr_round_slots = tourney_slots_df[tourney_slots_df['Slot'].str.contains(current_round)]

    round_matchups = build_round_matchups(tourney_seeds_df,curr_round_slots)
    matchups.append(round_matchups)
    current_round_bracket = build_game_matchups(round_matchups, team_stats_season[2023].groupby('TeamID').last().reset_index())
    current_round_bracket = predict_bracket_winners(current_round_bracket, model, scalar)
    tourney_seeds_df = update_seeds_with_winners(current_round_bracket, tourney_seeds_df)
    


In [ ]:
complete_bracket = pd.concat(matchups)
complete_bracket.reset_index(drop=True, inplace=True)
# complete_bracket.to_csv('2023/MNCAATourneyPredictions_matchup_2023.csv', index=False)
complete_bracket

# tourney_seeds_df
# test_dict = {}
# test_dict[type(lgr).__name__] = [accuracy_score(y_test, y_pred)]
# test_dict

In [ ]:
%cd ../../../

In [ ]:
import train_save_load_all_models as tsm
import os


models = tsm.load_models()
for model_name, model in models['avg_10'].items():
    print(model_name)
    print(model)



